In [6]:
import pandas as pd
import os
from datetime import time, datetime, timedelta


In [7]:
import os

def list_files_in_folder(folder_path):
    file_paths = []
    
    # Check if the folder exists
    if os.path.exists(folder_path):
        # Iterate through all files in the folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            
            # Check if the path is a file (not a directory)
            if os.path.isfile(file_path):
                file_paths.append(file_path)
    
    return file_paths

folder_path = 'Aylesford Data'  # Replace with the actual folder path
files_path = list_files_in_folder(folder_path)

# Print the list of file paths
for file_path in files_path:
    print(file_path)
    


Aylesford Data\WR Aylesford DC (AMBIENT-DB-TSA-A_B_AMBIENT-DB-TSA-A_B) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (AMBIENT-DOCK24_AMBIENT-DOCK24) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (AMBIENT-DOCK25_AMBIENT-DOCK25) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (BR-TS1_Battery Room Trailer Supply 1) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (BR-TS2_Battery Room Trailer Supply 2) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (CHIILED-WH-DB1&2-VAN_CHIILED-WH-DB1&2-VAN) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (CHILLED-WH-DB3_SW9-TRSKTS_CHILLED-WH-DB3_SW9-TRSKTS) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (CHILLED-WH-DBOSC-TRSKTS_CHILLED-WH-DBOSC-TRSKTS) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (ER-TSF_Engine Room Trailer Supply 2) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesford DC (ER-TSM_Engine Room Trailer Supply 1) 2024-03-11_2024-03-31.csv
Aylesford Data\WR Aylesf

In [8]:
def update_and_save_dataframe(file_path, dfs_consumption, dfs_detail):
    # Read CSV into a DataFrame
    df = pd.read_csv(file_path)
    
    # Convert DateTime to datetime format with day first
    df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True)
    
    # Set DateTime as the index
    df.set_index('DateTime', inplace=True)
    
    # Ensure that there is an entry for 23:59 on the last day
    last_date = df.index.date[-1]
    end_of_day = pd.Timestamp.combine(last_date, time(23, 59))
    if end_of_day not in df.index:
        # If there's no data at all for the last day, create a row with 0 consumption
        if not (df.index.date == last_date).any():
            df.loc[end_of_day] = [0 for _ in range(len(df.columns))]  # or appropriate default values
        else:
            df.loc[end_of_day] = df.loc[df.index.date == last_date].iloc[-1]
        df.sort_index(inplace=True) # Sort the index after adding the new row
    
    # Resample to 5 minute intervals and interpolate the kWh values
    df_resampled = df.resample('5T').ffill()
    
    # Divide the kWh values by 6 to distribute them across the 5-minute intervals
    df_resampled[' kWh'] /= 6
    
    # Rename the DateTime index back to a column
    df_resampled.reset_index(inplace=True)
    
    # Convert DateTime to the desired format with added seconds
    df_resampled['DateTime'] = df_resampled['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Add two additional columns: date and time
    df_resampled['date'] = df_resampled['DateTime'].str.split(" ").str[0]
    df_resampled['time'] = df_resampled['DateTime'].str.split(" ").str[1]
    
    # Rename columns
    df_resampled.columns = ['date_time', 'meter_desc', 'consumption', 'date', 'time']
    
    # Map meter_desc to meter_id
    meter_id_mapping = {
        "CHIILED-WH-DB1&2-VAN_CHIILED-WH-DB1&2-VAN": '10001',
        "CHILLED-WH-DB3/SW9-TRSKTS_CHILLED-WH-DB3/SW9-TRSKTS": '10002',
        "CHILLED-WH-DBOSC-TRSKTS_CHILLED-WH-DBOSC-TRSKTS": '10003',
        "FRV-WH-TRLSKTS_FRV-WH-TRLSKTS": '20001',
        "AMBIENT-DB-TSA-A/B_AMBIENT-DB-TSA-A/B": '30001',
        "AMBIENT-DOCK25_AMBIENT-DOCK25": '40001',
        "AMBIENT-DOCK24_AMBIENT-DOCK24": '40002',
        "ER-TSM_Engine Room Trailer Supply 1": '50001',
        "ER-TSF_Engine Room Trailer Supply 2": '50002',
        "BR-TS1_Battery Room Trailer Supply 1": '60001',
        "BR-TS2_Battery Room Trailer Supply 2": '60002',
        "DB 1-12 Sitewide Trailer Hookup Points_DB 1-12 Sitewide Trailer Hookup Points": '00001',
        "Trailer Hook Up Points BusBar East_Trailer Hook Up Points BusBar East": '00002',
        "Trailer Hook Up Points BusBar West_Trailer Hook Up Points BusBar West": '00003'
    }
    df_resampled['meter_id'] = df_resampled['meter_desc'].map(meter_id_mapping)
    
    # Add a new mapping for 'Site' based on 'meter_desc'
    site_mapping = {
        "CHIILED-WH-DB1&2-VAN_CHIILED-WH-DB1&2-VAN": 'Aylesford',
        "CHILLED-WH-DB3/SW9-TRSKTS_CHILLED-WH-DB3/SW9-TRSKTS": 'Aylesford',
        "CHILLED-WH-DBOSC-TRSKTS_CHILLED-WH-DBOSC-TRSKTS": 'Aylesford',
        "FRV-WH-TRLSKTS_FRV-WH-TRLSKTS": 'Aylesford',
        "AMBIENT-DB-TSA-A/B_AMBIENT-DB-TSA-A/B": 'Aylesford',
        "AMBIENT-DOCK25_AMBIENT-DOCK25": 'Aylesford',
        "AMBIENT-DOCK24_AMBIENT-DOCK24": 'Aylesford',
        "ER-TSM_Engine Room Trailer Supply 1": 'Aylesford',
        "ER-TSF_Engine Room Trailer Supply 2": 'Aylesford',
        "BR-TS1_Battery Room Trailer Supply 1": 'Aylesford',
        "BR-TS2_Battery Room Trailer Supply 2": 'Aylesford',
        "DB 1-12 Sitewide Trailer Hookup Points_DB 1-12 Sitewide Trailer Hookup Points": 'Leyland',
        "Trailer Hook Up Points BusBar East_Trailer Hook Up Points BusBar East": 'Leyland',
        "Trailer Hook Up Points BusBar West_Trailer Hook Up Points BusBar West": 'Leyland'
    }

    # Use the new mapping to create the 'Site' column
    df_resampled['Site'] = df_resampled['meter_desc'].map(site_mapping)
    
    # Map meter_desc to site_id
    site_id_mapping = {
        "CHIILED-WH-DB1&2-VAN_CHIILED-WH-DB1&2-VAN": '961',
        "CHILLED-WH-DB3/SW9-TRSKTS_CHILLED-WH-DB3/SW9-TRSKTS": '961',
        "CHILLED-WH-DBOSC-TRSKTS_CHILLED-WH-DBOSC-TRSKTS": '961',
        "FRV-WH-TRLSKTS_FRV-WH-TRLSKTS": '961',
        "AMBIENT-DB-TSA-A/B_AMBIENT-DB-TSA-A/B": '961',
        "AMBIENT-DOCK25_AMBIENT-DOCK25": '961',
        "AMBIENT-DOCK24_AMBIENT-DOCK24": '961',
        "ER-TSM_Engine Room Trailer Supply 1": '961',
        "ER-TSF_Engine Room Trailer Supply 2": '961',
        "BR-TS1_Battery Room Trailer Supply 1": '961',
        "BR-TS2_Battery Room Trailer Supply 2": '961',
        "DB 1-12 Sitewide Trailer Hookup Points_DB 1-12 Sitewide Trailer Hookup Points": '775',
        "Trailer Hook Up Points BusBar East_Trailer Hook Up Points BusBar East": '775',
        "Trailer Hook Up Points BusBar West_Trailer Hook Up Points BusBar West": '775'
    }
    
    df_resampled['site_id'] = df_resampled['meter_desc'].map(site_id_mapping)
    # Create t_novus_meter_consumption DataFrame
    df_consumption = df_resampled[['consumption', 'date_time', 'meter_id']].copy()
    dfs_consumption.append(df_consumption)

    # Create t_novus_meter_consumption_detail DataFrame
    df_detail = df_resampled[['consumption', 'date', 'date_time', 'meter_id', 'time']].copy()
    dfs_detail.append(df_detail)

In [9]:
# Initialize lists to store DataFrames for each type
dfs_consumption = []
dfs_detail = []

for each_file in files_path:
    update_and_save_dataframe(each_file, dfs_consumption, dfs_detail)

# Concatenate and save DataFrames for consumption and details
df_consumption_final = pd.concat(dfs_consumption, ignore_index=True)
df_detail_final = pd.concat(dfs_detail, ignore_index=True)
df_detail_final = df_detail_final[['time', 'consumption', 'meter_id', 'date', 'date_time']]

#df_consumption_final.to_csv('t_checkit_meter_consumption_aylesford_leyland.csv', index=False)
#df_detail_final.to_csv('t_checkit_meter_consumption_detail_aylesford_leyland.csv', index=False)



C:\Users\DingoLuo\AppData\Local\Temp\ipykernel_21176\889050708.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df.resample('5T').ffill()
C:\Users\DingoLuo\AppData\Local\Temp\ipykernel_21176\889050708.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df.resample('5T').ffill()
C:\Users\DingoLuo\AppData\Local\Temp\ipykernel_21176\889050708.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df.resample('5T').ffill()
C:\Users\DingoLuo\AppData\Local\Temp\ipykernel_21176\889050708.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df.resample('5T').ffill()
C:\Users\DingoLuo\AppData\Local\Temp\ipykernel_21176\889050708.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, plea

In [11]:
from datetime import datetime
import os

# Define the month and year for filtering
target_month = 3  # Replace with the desired month
target_year = 2024  # Replace with the desired year

# Convert the 'date_time' column to datetime format
df_detail_final['date_time'] = pd.to_datetime(df_detail_final['date_time'], format='%Y-%m-%d %H:%M:%S')

# Filter the data based on the given month and year
filtered_df = df_detail_final[(df_detail_final['date_time'].dt.month == target_month) & 
                              (df_detail_final['date_time'].dt.year == target_year)]

# Extract the month and year for the output filename
output_file_suffix = f'_{target_year}_{target_month:02d}'

# Define the output directory path
output_directory = 'output/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Define the output file path within the output directory
output_file_path = os.path.join(output_directory, f't_checkit_meter_consumption_detail{output_file_suffix}.csv')

# Save the filtered DataFrame to the output file
filtered_df.to_csv(output_file_path, index=False)

print(f"Filtered data saved to {output_file_path}")


Filtered data saved to output/t_checkit_meter_consumption_detail_2024_03.csv
